In [1]:
%load_ext autoreload
%autoreload 2
import wandb as wb
from clap import Clap, ClapDataset, ClapTrainer, SymmetricCrossEntropyLoss, get_target_device, load_config
from torch.utils.data import DataLoader
from torch import optim

In [2]:
config_path = "clap/configs/clap_htsat-tiny_gpt2.yml"
config = load_config(config_path)
device = get_target_device()

In [9]:
# change these accordingly
seed = ClapTrainer.set_random_seed(None)
train_dataset = ClapDataset(config=config_path, kind="train")
val_dataset = ClapDataset(config=config_path, kind="val")
test_dataset = ClapDataset(config=config_path, kind="test")

Random seed set as 2831676980


In [4]:
wb.login()

wandb: Currently logged in as: leonakkad. Use `wandb login --relogin` to force relogin


True

In [5]:
wb.init(
    # set the wandb project where this run will be logged 
    project='Custom-CLAP',
    name="Test run with hdf5 file",
    # track hyperparameters
    config=config
)
config = wb.config

In [10]:
# define data loaders
train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config["batch_size"])
test_loader = DataLoader(test_dataset, batch_size=config["batch_size"])

In [11]:
# define model, optimize and loss function
model = Clap(config).to(device)
print(f"Number of parameters to train: {sum(p.numel() for p in model.parameters())}")
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
loss_fn = SymmetricCrossEntropyLoss()
trainer = ClapTrainer(train_loader, val_loader, test_loader, model, optimizer, loss_fn, config["epochs"])

C:\Users\leon\miniconda3\envs\custom-clap\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Number of parameters to train: 167871888


In [12]:
metrics = trainer.train_and_eval("checkpoints/test.ckpt", None, False)


Starting to train Model


Training epoch 1 (lr=array([0.0001])):  31%|███▏      | 314/1002 [48:39<1:46:36,  9.30s/it]


KeyboardInterrupt: 

In [13]:
wb.finish()

train/a2t/batch recall@1,▁▁▁▁▂▂▂▁▃▂▂▂▃▃▂▃▅▄▄▅▅▆▃▆▆▆▆▇▆▅▅▆▅▅▇█▄▇█▆
train/a2t/batch recall@10,▁▁▁▁▂▂▄▁▃▃▄▄▄▅▄▅▆▆▆▅▅▅▆▇▆▆▆▆▇▆▆▇▇▅▇▇█▇█▇
train/a2t/batch recall@5,▁▁▂▁▂▂▃▁▃▃▃▃▃▅▄▄▅▅▅▅▅▅▅▆▆▅▆▅▇▇▆▇▆▅▇▇▇▇█▇
train/batch loss,████▇▇▆█▆▇▆▅▆▅▅▅▄▄▄▅▄▄▄▄▃▄▃▃▂▃▃▂▃▅▂▃▂▂▁▂
train/step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/t2a/batch recall@1,▁▁▁▁▁▁▂▁▂▃▃▄▃▄▃▃▄▃▃▄▄▄▃▃▆▆▇▄▇▆▅▆▅▃▆▆▅▇▇█
train/t2a/batch recall@10,▁▂▁▁▂▃▄▁▃▄▄▄▄▅▅▄▅▆▅▅▆▅▆▆▆▆▇▅▇▇▇▇▇▅▇▆▇▇█▇
train/t2a/batch recall@5,▁▂▁▂▂▃▃▁▂▃▃▃▃▄▄▃▅▆▅▄▅▅▅▆▆▅▇▅▇▅▇▆▇▅▇▇▇▇█▇
train/a2t/batch recall@1,0.23438
train/a2t/batch recall@10,0.79688
train/a2t/batch recall@5,0.64062
